# Example for usage ARc Policy via a Strands Agent

Install requirements

In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [ ]:
# %pip install -r requirements.txt --upgrade

In [ ]:
# If needed, this will allow for the restart of the notebook

# from IPython.display import display_html
# def restart_kernel():
#     display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
# restart_kernel()


In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.40.50
botocore==1.40.76
opentelemetry-instrumentation-boto==0.60b1
bedrock-agentcore==0.1.7
bedrock-agentcore-starter-toolkit==0.1.24


In [2]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

## Setup for the agent

### Create Code for the Agent
Create agents folder if it's not created.

In [3]:
![ ! -d "agents" ] && mkdir agents

In [10]:
%%writefile agents/claimvalidation-agent-3.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
from strands_tools import retrieve
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3
import requests
import time
from boto3.session import Session


# Get boto session
boto_session = Session()

from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException



# Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used



# AWS Configuration
# AWS_REGION = "us-east-1"
# MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

# Initialize Bedrock client to verify connectivity
# bedrock_client = boto3.client(
#     service_name="bedrock-runtime",
#     region_name=AWS_REGION
# )

# print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
# print(f"✓ Using model: {MODEL_ID}")

# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:161615149547:automated-reasoning-policy/malxiyr0ojy2"
GUARDRAIL_ID = "an852wptcjol"
GUARDRAIL_VERSION = "4"
KNOWLEDGE_BASE_ID = "CZDJXI9C4E"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
# MODEL_ID = "us.amazon.nova-lite-v1:0"
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID



# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        print ("HERE LOOKIE HERE",content_to_validate)
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = contentbedrock_model
            new_output = new_output + f"\n\nfindings: {self.findings}"
            new_output = new_output + f"\n\nclaim_valid: {self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""


# Create agent with the guardrail-protected model
agent = Agent(
    name="Claims Validator - ARC",
    description="A Single agent with Claims Validation tools capabilities",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve],
    system_prompt=agent_instructions
)

####### Agent Core ##################
app = BedrockAgentCoreApp()

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()
####### Agent Core ##################

################# A2A ################
# app = FastAPI()
# runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
# host, port = "0.0.0.0", 9000

# a2a_server = A2AServer(
#     agent=agent,
#     http_url=runtime_url,
#     serve_at_root=True,
    
# )

# @app.get("/ping")
# def ping():
#     return {"status": "healthy"}

# # @app.on_event("startup")
# # async def startup_event():
# #     """Initialize MCP client on startup"""
# #     await setup_agent_tools()

# app.mount("/", a2a_server.to_fastapi_app())



# if __name__ == "__main__":
#     uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/claimvalidation-agent-3.py


Let's write a requirements.txt file with dependencies that are needed for the agent.

###  Deploy to AgentCore Runtime

##### Configure and deploy our  agent:

In [11]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime_claimvalidation_agent = Runtime()
claimvalidation_agent_name="aws_claimvalidation_assistant"

region = boto_session.region_name

# Configure the deployment
response_claimvalidation_agent = agentcore_runtime_claimvalidation_agent.configure(
    entrypoint="agents/claimvalidation-agent-3.py",
    # execution_role=execution_role_arn_claimvalidation,
    # execution_role=execution_role_arn,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=claimvalidation_agent_name,
    # authorizer_configuration={
    #     "customJWTAuthorizer": {
    #         "allowedClients": [cognito_config.get("client_id")],
    #         "discoveryUrl": cognito_config.get("discovery_url"),
    #     }
    # },
    # protocol="A2A",
)

print("Configuration completed:", response_claimvalidation_agent)




Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/claimvalidation-agent-3.py, bedrock_agentcore_name=claimvalidation-agent-3
Memory configured with STM only
Configuring BedrockAgentCore agent: aws_claimvalidation_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
Found existing memory ID from previous launch: aws_claimvalidation_assistant_mem-J7Hg4Z84cG


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Keeping 'aws_claimvalidation_assistant' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role=None ecr_repository=None auto_create_ecr=True memory_id=None


In [12]:
launch_result_claimvalidation = agentcore_runtime_claimvalidation_agent.launch()
print("Launch completed:", launch_result_claimvalidation.agent_arn)

claimvalidation_agent_arn = launch_result_claimvalidation.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: aws_claimvalidation_assistant
✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_claimvalidation_assistant_mem-J7Hg4Z84cG...
  Found memory: aws_claimvalidation_assistant_mem-J7Hg4Z84cG
Found existing memory in cloud: aws_claimvalidation_assistant_mem-J7Hg4Z84cG
Existing memory has 0 strategies
✅ Using existing STM-only memory
Starting CodeBuild ARM64 deployment for agent 'aws_claimvalidation_assistant' to account 161615149547 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating E

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_claimvalidation_assistant


Getting or creating CodeBuild execution role for agent: aws_claimvalidation_assistant
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-1db4e090aa
Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-1db4e090aa
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: aws_claimvalidation_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-aws_claimvalidation_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.3s
🔄 DOWNLOAD_SOURCE started (total: 8s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 10s)
✅ BUILD completed in 15.4s
🔄 POST_BUILD started (total: 26s)
✅ POST_BUILD completed in 12.4s
🔄 COMPLETED started (total: 38s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfu

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/aws_claimvalidation_assistant-JahQJ5DGNQ


In [13]:
status_response = agentcore_runtime_claimvalidation_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_claimvalidation_assistant_mem-J7Hg4Z84cG...
  Found memory: aws_claimvalidation_assistant_mem-J7Hg4Z84cG
Retrieved Bedrock AgentCore status for: aws_claimvalidation_assistant


Final status: READY


### Invoking AgentCore Runtime

#### Test agents

Now, let's invoke the  agent

In [21]:
from IPython.display import Markdown, display
import json

invoke_response = agentcore_runtime_claimvalidation_agent.invoke({"prompt": "I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?"})
display(Markdown(invoke_response['response'][0]))

"Based on the policy information I have access to and your description of the circumstances, I can provide you with a preliminary assessment:\n\n**This Claim is Invalid and no appraisal nor settlement is required**\n\n**Reasons for Claim Invalidity:**\n\n1. **Business Use Exclusion Violation**: The policy explicitly excludes coverage when the vehicle is being used \"for retail or wholesale delivery, including, but not limited to, the pickup, transport or delivery of magazines, newspapers, mail or food.\" Your use of the personal vehicle to deliver goods for your company falls directly under this exclusion.\n\n2. **Commercial Activity Exclusion**: The policy states coverage will not apply when the vehicle is being used \"to carry persons or property for compensation or a fee.\" As a company employee delivering goods, you were likely being compensated for this business activity.\n\n**Regarding the 3-month reporting delay**: While the policy requires prompt reporting, late reporting alone doesn't automatically i

In [20]:

response_text = invoke_response['response'][0]


"I understand you're asking about coverage for an accident that occurred while delivering goods for your company over 3 months ago. However, as an automotive claims validation specialist, I need the complete claim information in JSON format to make an official determination.\n\nTo properly validate your claim, please provide:\n\n```json\n{\n  \"claim_number\": \"your_claim_number\",\n  \"policy_number\": \"your_policy_number\",\n  \"date_of_loss\": \"YYYY-MM-DD\",\n  \"date_claim_filed\": \"YYYY-MM-DD\",\n  \"vehicle_info\": {\n    \"make\": \"vehicle_make\",\n    \"model\": \"vehicle_model\", \n    \"year\": \"vehicle_year\",\n    \"vin\": \"vehicle_vin\"\n  },\n  \"incident_details\": {\n    \"location\": \"accident_location\",\n    \"time\": \"time_of_accident\",\n    \"description\": \"what_happened\"\n  },\n  \"damage_info\": {\n    \"description\": \"damage_description\",\n    \"estimated_cost\": \"repair_estimate\"\n  },\n  \"driver_info\": {\n    \"name\": \"driver_name\",\n    \"license_number\": \"l